In [ ]:
import pandas as pd
import pyarrow
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
## Import 
import numpy as np
import pandas as pd
import datetime as dt
import joblib
from collections import defaultdict
import json
import datetime as dt
from pathlib import Path

# To display BSNs fully
pd.set_option("display.max_colwidth", 1000)

# For convenience
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
from wpi_uitkeringsfraude.project_paths import ARTIFACT_PATH, DATA_PATH, CONFIG_PATH, INFO_PATH
from wpi_uitkeringsfraude.model.manage_model_info import load_feature_list
# from wpi_uitkeringsfraude.model.build_model import filter_application_handling
from wpi_uitkeringsfraude.settings.settings import WPISettings
# from wpi_uitkeringsfraude.components import SocratesDienstPersoonJoin, SocratesAdresFeatures
from wpi_uitkeringsfraude.scorer import Scorer

In [ ]:
df = pd.read_excel("\data\pilot\Pilot_BiasData20240110.xlsx")
# df = pd.read_excel("\data\pilot\Pilot_BiasData20240110_fp.xlsx", engine='calamine')
# df = pd.read_csv("\data\pilot\Pilot_BiasData20240110.csv", delimiter=";", encoding='latin-1', dtype={'SlimmeCheck.VALUE_TOTAL_VERMOGEN':float})
# , decimal=",")

In [ ]:
df.loc[
       # (df['Slimme Check'] == 'Nee')
       # & 
       (~df['Sherlock.Resultaat onderzoek'].isna())]

In [ ]:
pd.set_option("display.precision", 15)

In [ ]:
df.loc[(df['Index'] == 942), 'SlimmeCheck.VALUE_TOTAL_VERMOGEN']

In [ ]:
df.head()

In [ ]:
float(str(-141.5599979439524))

In [ ]:
# set the max columns to none
pd.set_option('display.max_columns', None)

In [ ]:
# We only want the entries that have a result from HH
df = df.loc[~df['Sherlock.Resultaat onderzoek'].isna()]

In [ ]:
# Because we have no date of aanvraag in this df, we synthisize one halfway of the pilot period
# So 15/7/2023
df['dtaanvraag'] = pd.Timestamp('2023-07-15')

In [ ]:
# Label each value (afboekcode) in Sherlock.Resultaat onderzoek with 0 (niet-onderzoekswaardig) or 1 (onderzoekswaardig) 
# These labels are defined based on Table 1 in the pre-pilot Bias Analysis documentation and after discussion with Jochem and Roland (handhaving) 
map_results_handhaving = {
    'Geen wijziging (1)': '0',
    'Afwijzing: geen medewerking (21)': '1',
    'Niet onderzoekswaardig (0)': '0',
    'Afwijzing: woon/leefsituatie (23)': '1',
    'Geen wijziging (Rechtvaardigheid)': '1',
    'Onjuiste opvoer (100)': '0',
    'Afwijzing: voorliggende voorziening': '1',
    'Wijziging: middelen (12)': '1',
    'Wijziging: woon/leefsituatie (13)': '1',
    'Onjuiste overdracht: retour': '0',
    'Afwijzing: middelen (22)': '1',
    'Niet onderzoekswaardig vooronderzoek (0)': '0'
}
# Create new column with results handhaving onderzoek 
df['Label'] = df['Sherlock.Resultaat onderzoek'].map(map_results_handhaving)

In [ ]:
# We want onderzoekswaardig to be differently mapped
dict_map_onderzoekswaardig = {'Onderzoekswaardig' : 1,
                              'Niet onderzoekswaardig' : 0}

df['onderzoekswaardig'] = df['Label'].replace(dict_map_onderzoekswaardig)

df['onderzoekswaardig'].value_counts()

In [ ]:
# # For the verrijking, only the dienstnummer and date of aanvraag are required
# pilot_data_verrijking_20240125 = df_pilot[['SlimmeCheck.APPLICATION_DIENSTNR', 'dtaanvraag']]

# # Change column names to be equal to other datasets for verrijking
# columns_verrijking = ['application_dienstnr', 'dtaanvraag']
# pilot_data_verrijking_20240125.columns = columns_verrijking

# # Save to excel
# # pilot_data_verrijking_20240125.to_csv('20240111_pilot_data_verrijking.csv', index=False)

In [ ]:
# Sample list with initial columns
entries = ['SlimmeCheck.VALUE_AFSPRAKEN_NO_CONTACT_COUNT_LAST_YEAR',
       'SlimmeCheck.VALUE_APPLIED_FOR_SAME_PRODUCT_LAST_YEAR',
       'SlimmeCheck.VALUE_TOTAL_VERMOGEN',
       'SlimmeCheck.VALUE_DAYS_SINCE_LAST_DIENST_END',
       'SlimmeCheck.VALUE_DAYS_SINCE_LAST_RELOCATION',
       'SlimmeCheck.VALUE_AT_LEAST_ONE_ADDRESS_IN_AMSTERDAM',
       'SlimmeCheck.VALUE_AFSPRAKEN_NO_SHOW_COUNT_LAST_YEAR',
       'SlimmeCheck.VALUE_RECEIVED_SAME_PRODUCT_LAST_YEAR',
       'SlimmeCheck.VALUE_DEELNAMES_STARTED_PERCENTAGE_LAST_YEAR',
       'SlimmeCheck.VALUE_ACTIVE_ADDRESS_COUNT',
       'SlimmeCheck.VALUE_HAS_PARTNER', 'SlimmeCheck.VALUE_HAS_MEDEBEWONER',
       'SlimmeCheck.VALUE_AVG_PERCENTAGE_MAATREGEL',
       'SlimmeCheck.VALUE_SUM_INKOMEN_BRUTO_VALUE',
       'SlimmeCheck.VALUE_SUM_INKOMEN_BRUTO_WAS_MEAN_IMPUTED']

# Creating a dictionary with original entries as keys and lowercase values after the dot
result_dict = {entry: entry.split('VALUE_')[1].lower() for entry in entries}

# Printing the resulting dictionary
print(result_dict)

In [ ]:
# We need the column names to be the same as when the model was trained for the prediction
df = df.rename(columns = 
   result_dict
)

In [ ]:
# We need the column names to be the same as when the model was trained for the prediction
df = df.rename(columns = 
   {"sum_inkomen_bruto_value":"sum_inkomen_bruto"}
)

In [ ]:
df['onderzoekswaardig'] = df['onderzoekswaardig'].astype(int)

In [ ]:
df['total_vermogen'].info()

In [ ]:
df['total_vermogen'].dtype

In [ ]:
for column in df.columns:
    if df[column].dtype == 'object':
        print(column)

In [ ]:
df['total_vermogen'].info()

In [ ]:
df.loc[(df['Index'] == 942), 'total_vermogen']

In [ ]:
# float_but_not_float = ['total_vermogen', 'sum_inkomen_bruto', 'deelnames_started_percentage_last_year']
# for column in float_but_not_float:
#     df[column] = df[column].str.replace(",",".")
#     df[column] = pd.to_numeric(df[column])
#     # df[column] = df[column].convert_objects(convert_numeric=True)

In [ ]:
df.loc[(df['Index'] == 942), 'total_vermogen']

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show

# df_training = df_training.sort_values(by='dtaanvraag')

# X = df.loc[:, df.columns != 'onderzoekswaardig']
# y = df['onderzoekswaardig']

seed = 42
# np.random.seed(seed)
X_test, y_test = df.loc[:, df.columns != 'onderzoekswaardig'], df['onderzoekswaardig']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=seed)
# Load models
model_file_prepilot = Path('\Models\20220523_model_used_in_prepilot.pkl')
model_file_pilot = Path('\Models\20240228_wpi_model_pilot_31.pkl')
models = {'model_before_reweighing':model_file_prepilot,
          'model_after_reweighing': model_file_pilot}
# Set threshold
thr=0.63

for model_name, model_file in models.items():
    model_dict = joblib.load(model_file)
    model = model_dict["model"]
    prep = model[:-1]  # all but the last pipeline steps, hence all transformers, but not the model
    clf = model[-1]  # the actual model
    num_cols, cat_cols = load_feature_list()
    label = "onderzoekswaardig"
    # Model scores
    X_test[f"{model_name}_score"] = model.predict_proba(X_test)[:,1]
    # Check if above threshold
    X_test[f"{model_name}_prediction"] = (model.predict_proba(X_test)[:, 1] >= thr).astype('int')
models = {'model_before_reweighing':model_file_prepilot,
          'model_after_reweighing': model_file_pilot}


# X_train['onderzoekswaardig'] = y_train
X_test['onderzoekswaardig'] = y_test

from sklearn.metrics import confusion_matrix, precision_score

conf_before_reweigh = confusion_matrix(X_test['onderzoekswaardig'], X_test['model_before_reweighing_prediction'])
conf_after_reweigh = confusion_matrix(X_test['onderzoekswaardig'], X_test['model_after_reweighing_prediction'])

print("Conf matrix before reweighing")
display(conf_before_reweigh)

print("Conf matrix after reweighing")
display(conf_after_reweigh)

print("Precision before reweighing")
# display(conf_before_reweigh[0,0]/(conf_before_reweigh[0,0]+conf_before_reweigh[0,1]))
print(precision_score(X_test['onderzoekswaardig'], X_test['model_before_reweighing_prediction']))

print("Precision after reweighing")
# display(conf_after_reweigh[0,0]/(conf_after_reweigh[0,0]+conf_after_reweigh[0,1]))
print(precision_score(X_test['onderzoekswaardig'], X_test['model_after_reweighing_prediction']))


In [ ]:
df = X_test

In [ ]:
X_test.head()

In [ ]:
X_test[['SlimmeCheck.SCORE', 'model_after_reweighing_score']]

In [ ]:
# X_test[['SlimmeCheck.SCORE', 'model_after_reweighing_score']]

In [ ]:
df.loc[(df['Index'] == 942), 'total_vermogen']

In [ ]:
X_test = df

In [ ]:
X_test.loc[(X_test['Index'] == 942), 'total_vermogen']

In [ ]:
X_test

In [ ]:
X_test.loc[X_test['SlimmeCheck.SCORE'] == X_test['model_after_reweighing_score'], 'sum_inkomen_bruto'].unique()

In [ ]:
for feature in X_test.columns:
    X_test.loc[X_test['SlimmeCheck.SCORE'] == X_test['model_after_reweighing_score'], feature]

In [ ]:
# df['model_prob']

In [ ]:
df

In [ ]:
df[['SlimmeCheck.SCORE', 'model_after_reweighing_prediction']]

In [ ]:
df['Bias.Geslacht'].value_counts()

In [ ]:
df.to_excel("\data\processed_bias_data\20240131_pilot_processed.xlsx", index=False)